# Quantum Teleportation

- `Quantum teleportation` is a protocol where a sender transmits a qubit to a receiver by making use of a shared entangled quantum state along with two bits of classical communication.
- The entanglement is lost in the process

### Teleportation Circuit
![](images/quantum_teleportation.png)

### Results

```python
Angle of Rotations are: 	[0.38864361 2.9653579  3.1155445 ]
Quantum state to teleport: 	[-0.01586755-0.08656106j  0.20506465+0.97478402j]

Sending Circuit
0: ──Rot(0.39,2.97,3.12)────╭●──H─┤  State
1: ──H───────────────────╭●─╰X────┤  State
2: ──────────────────────╰X───────┤  State
Output state: 
[-0.00793378-0.04328053j  0.10253232+0.48739201j  0.10253232+0.48739201j
 -0.00793378-0.04328053j -0.00793378-0.04328053j -0.10253232-0.48739201j
 -0.10253232-0.48739201j -0.00793378-0.04328053j]

After measurement results:
State: 	[-0.01586755-0.08656106j  0.20506465+0.97478402j]
CX bit: 	0
CZ bit: 	0

Receiving Circuit
0: ──|Ψ⟩─┤  State
Received State: 	[-0.01586755-0.08656106j  0.20506465+0.97478402j]

Comparing states (whether all elemnts are equal): True

```.

In [119]:
import pennylane as qml
from pennylane import numpy as np

In [133]:
wires = 3
dev_1q = qml.device("lightning.qubit", wires=1)
dev_nq = qml.device("lightning.qubit", wires=wires)


@qml.qnode(dev_1q)
def get_random_qubit(theta):
    qml.Rot(*theta, wires=0)
    return qml.state()


@qml.qnode(dev_nq)
def sending_circuit(theta):
    """
    Circuit for teleportation.
    Deferring measurements and receiving operations
    =========================
    Params:
        theta: length 3 vector for preparing qubit for teleportation
    Returns:
        qml.state
    """
    # state preparation on first qubit
    qml.Rot(*theta, wires=0)

    # create an EPR pair
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[1, 2])

    # Teleportation routine
    ## process for sending the random state
    qml.CNOT(wires=[0, 1])
    # mx = qml.measure(wires=1)
    qml.Hadamard(wires=0)
    # mz = qml.measure(wires=0)

    ## process for receiving the random state
    # qml.cond(mx, qml.X)(wires=2)
    # qml.cond(mz, qml.Z)(wires=2)
    
    return qml.state()


def measuring_states(state):
    """
    Taking the 3-qubit state and measuring the 1st, 2nd qubit
    """
    # probalilities
    probs = np.abs(state) ** 2
    z_probs = [sum(i).item() for i in (probs[:4], probs[4:])]
    x_probs = [sum(i).item() for i in (probs[[0, 1, 4, 5]], probs[[2, 3, 6, 7]])]

    # measurement
    mz = np.random.choice([0, 1], p=z_probs)
    mx = np.random.choice([0, 1], p=x_probs)

    # modifying the 3-q state to 1-q state
    if mz == 0:
        state = state[:4]
    else:
        state = state[4:]
    
    if mx == 0:
        state = state[:2]
    else:
        state = state[2:]
    
    # normalizing the measured state
    state = state / sum(np.abs(state)**2) ** 0.5
    return state, mx, mz


@qml.qnode(dev_1q)
def receiving_circuit(state, mx, mz):
    """
    Teleportation circuit at the receiving end.
    =========================
    Params:
        state: 1-q state
        mx: measurement of the EPR pair in sending circuit. used for controlling the X gate
        mz: measurement of the random-state-qubit in sending circuit. used for controlling the Z gate
    Returns:
        qml.state
    """
    # preparing initial state
    qml.QubitStateVector(state, wires=0)
    if mx:
        qml.X(wires=0)
    if mz:
        qml.Z(wires=0)
    return qml.state()

In [175]:
# Generating a random rotation
theta = np.pi * np.random.random(size=(3,))

init_state = get_random_qubit(theta)
print(f"Angle of Rotations are: \t{theta}")
print(f"Quantum state to teleport: \t{init_state}")

print("\nSending Circuit")
state = sending_circuit(theta)
print(qml.draw(sending_circuit)(theta))
print(f"Output state: \n{state}")

state, mx, mz = measuring_states(state)
print("\nAfter measurement results:")
print(f"State: \t{state}")
print(f"CX bit: \t{mx}")
print(f"CZ bit: \t{mz}")

print("\nReceiving Circuit")
state = receiving_circuit(state, mx, mz)
print(qml.draw(receiving_circuit)(state, mx, mz))
print(f"Received State: \t{state}")

compare = ((state - init_state) < 1e-10).all()
print(f"\nComparing states (whether all elements are equal): {compare}")

Angle of Rotations are: 	[0.38864361 2.9653579  3.1155445 ]
Quantum state to teleport: 	[-0.01586755-0.08656106j  0.20506465+0.97478402j]

Sending Circuit
0: ──Rot(0.39,2.97,3.12)────╭●──H─┤  State
1: ──H───────────────────╭●─╰X────┤  State
2: ──────────────────────╰X───────┤  State
Output state: 
[-0.00793378-0.04328053j  0.10253232+0.48739201j  0.10253232+0.48739201j
 -0.00793378-0.04328053j -0.00793378-0.04328053j -0.10253232-0.48739201j
 -0.10253232-0.48739201j -0.00793378-0.04328053j]

After measurement results:
State: 	[-0.01586755-0.08656106j  0.20506465+0.97478402j]
CX bit: 	0
CZ bit: 	0

Receiving Circuit
0: ──|Ψ⟩─┤  State
Received State: 	[-0.01586755-0.08656106j  0.20506465+0.97478402j]

Comparing states (whether all elements are equal): True
